# Carregando Pacotes

In [1]:
from transformers import pipeline
import torch
import pdfplumber
from transformers import AutoTokenizer

from sentence_transformers import SentenceTransformer
import numpy as np

In [2]:
# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

🔧 Rodando em: GPU


# Tokenização do Texto

In [7]:
# 1. Modelo de Embeddings (CPU/GPU)
modelo_embed = SentenceTransformer("all-MiniLM-L6-v2", device="cuda" if device == 0 else "cpu")

# 2. Vetoriza os chunks (use sua variável 'divisao_do_texto')
chunks_embeddings = modelo_embed.encode(divisao_do_texto)

# 3. Modelo Generativo (FLAN-T5 base - leve para CPU)
gerador = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device=device
)

NameError: name 'divisao_do_texto' is not defined

# Tratamento do Documento

In [ ]:
def carregar_documento(caminho_do_pdf):

    with pdfplumber.open(caminho_do_pdf) as pdf:
        texto = " ".join(
            page.extract_text() for page in pdf.pages 
            if page.extract_text()  # Ignora páginas sem texto
        )
        texto = " ".join(texto.split())  # Normaliza espaços
    return texto

## Divisao em Chunks

In [4]:
def dividir_em_chunks_tokenizados(texto):

    tokens = tokenizer.tokenize(texto)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

## Janela Deslizante

In [5]:
def sliding_window_tokenizados(texto):
    tokens = tokenizer.tokenize(texto)
    janelas = []
    
    for i in range(0, len(tokens), stride):
        janela = tokens[i:i + window_size]
        texto_janela = tokenizer.convert_tokens_to_string(janela)
        janelas.append(texto_janela)
        
        # Para se chegarmos ao final do texto
        if i + window_size >= len(tokens):
            break
    
    return janelas

## Selecionar Divisão do Texto

In [6]:
#Divisao em Janela Deslizante

divisao_do_texto = sliding_window_tokenizados(documento)

#Divisao em Chunks

#divisao_do_texto = dividir_em_chunks_tokenizados(documento)

NameError: name 'documento' is not defined

# Implementação do RAG

In [8]:
def buscar_chunks_semanticos(pergunta, chunks, chunks_embeddings, top_k=3):
    """Encontra os chunks mais relevantes usando similaridade semântica."""
    pergunta_embed = modelo_embed.encode(pergunta)
    similaridades = np.dot(pergunta_embed, chunks_embeddings.T)
    top_indices = np.argsort(similaridades)[-top_k:][::-1]  # Pega os top-K mais relevantes
    return [chunks[i] for i in top_indices]



In [9]:
def gerar_resposta_rag(pergunta, chunks_relevantes):
    """Gera uma resposta natural com base nos chunks encontrados."""
    contexto = " ".join(chunks_relevantes)
    prompt = f"""
    Com base no contexto abaixo, responda de forma clara e concisa:
    Contexto: {contexto}
    Pergunta: {pergunta}
    Resposta:
    """
    resposta = gerador(prompt, max_length=150)
    return resposta[0]['generated_text']